In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Testicular_Cancer/GSE42647'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Acute hypersensitivity of pluripotent testicular cancer-derived embryonal carcinoma to low-dose 5-aza deoxycytidine"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell line: NT2/D1'], 1: ['cell type: human ebryonal carcinoma']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if gene expression data is available
# The available information does not explicitly mention gene expression. Therefore,
# we assume it's not available.
is_gene_available = False

# Variable availability and data type conversion
# Exploring the provided sample characteristics data

sample_characteristics = {
    0: ['cell line: NT2/D1-R1'],
    1: ['cell type: human ebryonal carcinoma']
}

# 'Testicular_Cancer' variable availability
trait_row = 1 if len(set([item.split(': ')[1] for item in sample_characteristics[1]])) > 1 else None

# 'age' variable availability
age_row = None  # No age data available in the provided characteristics

# 'gender' variable availability
gender_row = None  # No gender data available in the provided characteristics

# Data type conversion functions
# 'Testicular_Cancer' is categorical, converting as binary: 1 for 'cancer', 0 for non-cancer (however, no such data found)

def convert_trait(value):
    if value is None:
        return None
    # Define specific mapping if details provided, otherwise, set conversion to None.
    value = value.split(': ')[1]
    return 1 if value.lower() in ['human ebryonal carcinoma'] else 0

# No age and gender data found

def convert_age(value):
    return None

def convert_gender(value):
    return None

# Saving cohort info
save_cohort_info('GSE42647', './preprocessed/Testicular_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Testicular_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Testicular_Cancer/trait_data/GSE42647.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
